<a href="https://colab.research.google.com/github/zjzsu2000/CMPE297_AdvanceDL_Project/blob/main/Data_Preprocessing/data_black_scholes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/gdrive")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
import pandas as pd
name = 'VCYT_option_sigma_TresuryRate'
option = pd.read_csv('/gdrive/My Drive/Data set/Option/Tresury_rate/'+name+'.csv')

In [ ]:
import io
import os
import numpy as np
from scipy.stats import norm

In [ ]:
def black_scholes_21_put(row):
    S = row.UnderlyingPrice
    X = row.Strike
    T = row.date_diff / 365
    r = (float) (row.treasury_rate)/ 100
    σ = row.sigma_21
    d1 = (np.log(S / X) + (r + (σ ** 2) / 2) * T) / (σ * (T ** .5))
    d2 = d1 - σ * (T ** .5)
    P  = norm.cdf(-d2) * X * np.exp(-r * T) - S * norm.cdf(-d1)
    return P


def black_scholes_21(row):
    S = row.UnderlyingPrice
    X = row.Strike
    T = row.date_diff / 365
    r =  (float) (row.treasury_rate)/ 100
    σ = row.sigma_21
    d1 = (np.log(S / X) + (r + (σ ** 2) / 2) * T) / (σ * (T ** .5))
    d2 = d1 - σ * (T ** .5)
    C = S * norm.cdf(d1) - X * np.exp(-r * T) * norm.cdf(d2)
    return C

In [ ]:
def main(row):
  df_option_treasury=row.copy()
  df_option_treasury.drop(df_option_treasury.loc[df_option_treasury['treasury_rate']=='.'].index, inplace=True)
  options_df_new_put = df_option_treasury[df_option_treasury.Type == 'put'].drop(['Type'], axis=1)
  options_df_new_call = df_option_treasury[df_option_treasury.Type == 'call'].drop(['Type'], axis=1)
  options_df_new_put.head(3)
  options_df_new_call['black_scholes_pred_21'] = options_df_new_call.apply(black_scholes_21, axis=1)
  options_df_new_put['black_scholes_pred_21'] = options_df_new_put.apply(black_scholes_21_put, axis=1)
  mse = lambda df, pred_col: np.sum(np.square(df[['Bid', 'Ask']].mean(axis=1) - df[pred_col])) / df.shape[0]
  mse(options_df_new_call, 'black_scholes_pred_21')
  med_abs_err = lambda df, pred_col: (np.abs(df[['Bid', 'Ask']].mean(axis=1) - df[pred_col])).iloc[df.shape[0]//2]
  med_abs_err(options_df_new_call, 'black_scholes_pred_21')
  options_df_new_call.to_csv(('/gdrive/My Drive/Data set/Option/call_put/'+name+'_call.csv'))
  options_df_new_put.to_csv(('/gdrive/My Drive/Data set/Option/call_put/'+name+'_put.csv'))
main(option)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys
